In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget http://millionsongdataset.com/sites/default/files/tasteprofile/sid_mismatches.txt
!unzip drive/MyDrive/train_triplets.txt.zip
!unzip drive/MyDrive/taste_profile_song_to_tracks.txt.zip

In [7]:
import numpy as np
import pandas as pd

In [8]:
songToTrackDf = pd.read_csv("./taste_profile_song_to_tracks.txt", names=["song", "track"], sep="\t")
songToTrackDf.dropna(inplace=True)
userSongCountDf = pd.read_csv("./train_triplets.txt", names=["user", "song", "count"], sep="\t")
mismatches = []
with open("sid_mismatches.txt", "r", encoding='UTF-8') as f:
    for line in f:
        mismatches.append((line[8:8+len(songToTrackDf["song"][0])], line[9+len(songToTrackDf["song"][0]):9+len(songToTrackDf["song"][0])+len(songToTrackDf["track"][0])]))
mismatchesDf = pd.DataFrame(mismatches, columns=["song", "track"])
songToTrackDf = songToTrackDf[(~songToTrackDf["track"].isin(mismatchesDf["track"])) & (~songToTrackDf["song"].isin(mismatchesDf["song"]))]
userSongCountDf = userSongCountDf[userSongCountDf["song"].isin(songToTrackDf["song"])]
userSongTrackCountDf = userSongCountDf.merge(songToTrackDf, how="left", on="song")
userSongTrackCountDf = userSongTrackCountDf[userSongTrackCountDf["count"] > 1]
uniqueSongs = userSongTrackCountDf["track"].unique()
tmp = userSongTrackCountDf[["user", "track"]]
userSongListDf = tmp.groupby("user")["track"].apply(list).reset_index(name='tracks')


In [74]:
#Given 1 users song history, generate similar song pairs.
def generate_pair(history):
  output = []
  for i in history:
    for j in history:
      if i>j: # this always places the song with larger track_id in front to avoid duplicates in global setting
        output.append((i,j))
  return np.array(output)

In [277]:
import random

def sample_negatives(pairs):
  n = len(pairs)
  for i in range(n):
    a = random.randint(0,n)
    b = random.randint(0,n)


In [373]:
dataset_x = pd.read_csv('drive/MyDrive/parsed_subset.csv')
tracks = dataset_x['track_id'].str[2:-1]

In [374]:
tracks

0       TRARRZU128F4253CA2
1       TRARRJL128F92DED0E
2       TRARRUZ128F9307C57
3       TRARRWA128F42A0195
4       TRARRPG12903CD1DE9
               ...        
9995    TRBBWSE128F9313963
9996    TRBBWDJ128F42595D5
9997    TRBBWNS128F426F760
9998    TRBBWIP128F4287400
9999    TRBBWKS128F424E84A
Name: track_id, Length: 10000, dtype: object

In [375]:
userSongTrackCountDf

,user,song,count,track
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,TRMHBXZ128F4238406
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBSUJE12A6D4F8CF5,2,TRPLAXZ128F4292406
13,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5,TRPTWGR128F1452734
32,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOIZAZL12A6701C53B,5,TRHNGRY128E0791670
35,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOKRIMP12A6D4F5DA3,5,TRYSBOR128F92CA258
...,...,...,...,...
45795078,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOOFYTN12A6D4F9B35,4,TRRGQKP128F1469E20
45795088,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOSQJWM12A6D4F79E0,6,TRHJZAG128F1460889
45795090,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOTFOAE12A6D4F4511,2,TRSYALP128F145FBF6
45795095,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOUHHHH12AF729E4AF,2,TRKUAEO128F933ABFC


In [376]:
temp = userSongTrackCountDf.set_index('track').join(tracks, how='inner')
temp

,user,song,count,track_id


In [370]:
tracks = dataset_x['track_id'].reset_index(name='tracks')
temp = userSongTrackCountDf.join(tracks, how='inner')
temp = temp[["user", "track"]]
temp = temp.groupby("user").filter(lambda x: len(x['track']) > 1) 
temp = temp.groupby("user")["track"].apply(list).reset_index(name='tracks')[["tracks"]].to_numpy()
users = np.squeeze(temp)

In [ ]:
features = dataset_x[['track_id','artist_familiarity','danceability','duration','end_of_fade_in','key',
         'key_confidence', 'loudness', 'mode', 'tempo', 'time_signature', 
         'time_signature_confidence']]
#[['segments_loudness_max_time','segments_loudness_max','segments_confidence','segments_pitches','segments_timbre']]
features['track_id'] = features['track_id'].str[2:-1]
features = features.set_index('track_id')

In [320]:
pairs = None
for user in users:
  if pairs is None:
    pairs = generate_pair(user)
  else:
    pairs = np.concatenate((pairs,generate_pair(user)),axis=0)

pairs = np.unique(pairs, axis=0)

In [326]:
features

,artist_familiarity,danceability,duration,end_of_fade_in,key,key_confidence,loudness,mode,tempo,time_signature,time_signature_confidence
track_id,,,,,,,,,,,
TRARRZU128F4253CA2,0.557460,0.0,148.74077,0.192,0.0,0.591,-9.636,1.0,124.059,4,0.372
TRARRJL128F92DED0E,0.626958,0.0,252.99546,0.514,1.0,0.429,-11.061,0.0,80.084,4,0.533
TRARRUZ128F9307C57,0.425724,0.0,78.02730,0.974,3.0,0.000,-24.140,1.0,54.874,4,0.000
TRARRWA128F42A0195,0.611495,0.0,163.63057,0.000,7.0,0.380,-5.795,1.0,77.150,3,0.369
TRARRPG12903CD1DE9,0.367255,0.0,199.99302,0.000,10.0,0.551,-16.477,1.0,120.382,4,1.000
...,...,...,...,...,...,...,...,...,...,...,...
TRBBWSE128F9313963,0.048853,0.0,237.16526,0.000,7.0,0.444,-6.177,1.0,142.978,4,0.243
TRBBWDJ128F42595D5,0.843527,0.0,142.23628,0.000,9.0,0.756,-14.050,0.0,94.838,4,0.536
TRBBWNS128F426F760,0.595661,0.0,346.51383,0.235,7.0,0.180,-5.802,1.0,88.344,5,0.307


In [325]:
features.loc[pairs[2][0]].to_numpy()

KeyError: ignored

In [319]:

datapoints = []
for pair in pairs:
  song1 = np.append(pairs[0][0],features.loc[pairs[0][0]].to_numpy())
  song2 = np.append(pairs[0][1],features.loc[pairs[0][1]].to_numpy())
  datapoints.append([song1,song2])

datapoints = np.asarray(datapoints)

KeyboardInterrupt: ignored

T
T


KeyError: ignored

In [278]:
datapoints[0].shape

(2, 12)

In [281]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [297]:
random.randint(0,10)

9

In [295]:
import random

n = len(datapoints)
output = []
for i in range(n):
  a = random.randint(0,n-1)
  b = random.randint(0,n-1)
  print(datapoints[a][0][0])
  if (datapoints[a][0][0] > datapoints[b][0][0]):
    output.append([datapoints[a][0],datapoints[b][0]])
  elif (datapoints[a][0][0] < datapoints[b][0][0]):
    output.append([datapoints[b][0],datapoints[a][0]])
  else:
    continue


output = np.asarray(output)
output.shape

Streaming output truncated to the last 5000 lines.
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB
TRAAKDG128F42A0ECB

(0,)

In [298]:
datapoints

array([[['TRAAKDG128F42A0ECB', '0.8331785608902065', '0.0', ...,
         '100.023', '4.0', '0.937'],
        ['TRAAGCZ128F93210FD', '0.8456018866402321', '0.0', ...,
         '105.206', '4.0', '1.0']],

       [['TRAAKDG128F42A0ECB', '0.8331785608902065', '0.0', ...,
         '100.023', '4.0', '0.937'],
        ['TRAAGCZ128F93210FD', '0.8456018866402321', '0.0', ...,
         '105.206', '4.0', '1.0']],

       [['TRAAKDG128F42A0ECB', '0.8331785608902065', '0.0', ...,
         '100.023', '4.0', '0.937'],
        ['TRAAGCZ128F93210FD', '0.8456018866402321', '0.0', ...,
         '105.206', '4.0', '1.0']],

       ...,

       [['TRAAKDG128F42A0ECB', '0.8331785608902065', '0.0', ...,
         '100.023', '4.0', '0.937'],
        ['TRAAGCZ128F93210FD', '0.8456018866402321', '0.0', ...,
         '105.206', '4.0', '1.0']],

       [['TRAAKDG128F42A0ECB', '0.8331785608902065', '0.0', ...,
         '100.023', '4.0', '0.937'],
        ['TRAAGCZ128F93210FD', '0.8456018866402321', '0.0', ...,
    